In [1]:
import pandas as pd
import numpy as np

In [120]:
import requests
from datetime import datetime
from timezonefinder import TimezoneFinder
import matplotlib.pyplot as plt
%matplotlib widget

to_dates = np.vectorize(datetime.fromtimestamp)
tzfinder = TimezoneFinder()

def get_openmeteo(latlong, start:datetime, stop:datetime):

    vars = [
        'shortwave_radiation',      #total sun for horizontal array 
        'direct_normal_irradiance', #used to calculate total sun for tilted array
        'diffuse_radiation',        #used to calculate total sun for tilited array (add to direct)
        'windspeed_10m',
        'winddirection_10m',
    ]

    #get timezone at coordinate so timestamps make sense
    timezone = tzfinder.timezone_at(lat=latlong[0], lng=latlong[1])

    forecast = {}
    for var in vars:

        response = requests.request(
            method = "GET", 
            url = "https://api.open-meteo.com/v1/forecast?", 
            params = {
                "latitude": latlong[0],
                "longitude": latlong[1],
                "hourly": var,
                "windspeed_unit": "ms",
                "start_date": start.strftime("%Y-%m-%d"),
                "end_date": stop.strftime("%Y-%m-%d"),
                "timeformat": "unixtime",
                "timezone": timezone
            }
        ).json()
        
        try:
            data = response['hourly']
        except:
            print(response)
        values = data[var]

        while values and values[-1] is None:    #remove trailing Nones
            values.pop()
        forecast[var] = values

        if('dates' not in forecast):            #if 
            forecast['timestamps'] = data['time'][:len(values)] #truncate timestamps to match values

    return forecast


get_openmeteo((40.11, -33), datetime(2022, 8, 22), datetime(2022, 8, 25))



# weather = {}
# for var in ['shortwave_radiation', 'direct_radiation', 'direct_normal_irradiance', 'diffuse_radiation']:

#     forecast = get_openmeteo((40.11, -33), var, datetime(2022, 8, 22), datetime(2022, 8, 25))

#     if 'dates' not in weather:
#         weather['dates'] = forecast['dates']
    
#     weather[var] = forecast['values']

# weather['sun_flat'] = weather['shortwave_radiation']
# weather['sun_tilt'] = weather['direct_normal_irradiance'] + weather['diffuse_radiation']

# plt.figure()
# plt.plot(weather['dates'], weather['sun_flat'], label='sun_flat')
# plt.plot(weather['dates'], weather['sun_tilt'], label='sun_tilt')

# plt.legend()


{'error': True, 'reason': 'Cannot initialize IconVariable from invalid String value diffuse_radiationwindspeed_10m for key hourly'}


KeyError: 'diffuse_radiationwindspeed_10m'